## Computational ressources

In [1]:
from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=28,name='make_profiles',walltime='00:30:00',job_extra=['--constraint=HSW24','--exclusive','--nodes=1'],memory='120GB',interface='ib0') 

cluster.scale(204) 

from dask.distributed import Client
client = Client(cluster)

client

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


Client Scheduler: tcp://172.30.100.4:41155 Dashboard: http://172.30.100.4:34215/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [2]:
!squeue -u albert7a

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           7482666      allc make_pro albert7a  R      26:32      1 n2523
           7482667      allc make_pro albert7a  R      26:32      1 n2636
           7482668      allc make_pro albert7a  R      26:32      1 n2113
           7482669      allc make_pro albert7a  R      26:32      1 n2114
           7482670      allc make_pro albert7a  R      26:32      1 n2744
           7482671      allc make_pro albert7a  R      26:32      1 n2745
           7482672      allc make_pro albert7a  R      26:32      1 n2960
           7482673      allc make_pro albert7a  R      26:32      1 n2961
           7482763      allc make_pro albert7a  R       0:00      1 n2060
           7482764      allc make_pro albert7a  R       0:00      1 n2746
           7482765      allc make_pro albert7a  R       0:00      1 n2759
           7482766      allc make_pro albert7a  R       0:00      1 n2760
           7482767      all

In [3]:
import time
nb_workers = 0
while True:
    nb_workers = len(client.scheduler_info()["workers"])
    if nb_workers >= 2:
        break
    time.sleep(1)
print(str(nb_workers)+' are ready !')

32 are ready !


In [4]:
!squeue -u albert7a

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           7482763      allc make_pro albert7a  R       0:18      1 n2060
           7482764      allc make_pro albert7a  R       0:18      1 n2746
           7482765      allc make_pro albert7a  R       0:18      1 n2759
           7482766      allc make_pro albert7a  R       0:18      1 n2760
           7482767      allc make_pro albert7a  R       0:18      1 n2761
           7482768      allc make_pro albert7a  R       0:18      1 n2762
           7482769      allc make_pro albert7a  R       0:18      1 n2944
           7482770      allc make_pro albert7a  R       0:18      1 n2945


## Librairies and customization

In [5]:
import xarray as xr 
import dask 
import numpy as np 
import os 
import time 
import glob
from datetime import date
today=date.today()

import sys
sys.path.insert(0,'/home/albert7a/git/xscale')
import xscale

import xscale.spectral.fft as xfft
from xscale.spectral.tools import plot_spectrum
import xscale.signal.generator as xgen

import matplotlib.pyplot as plt
params = {'figure.figsize' : (15, 8),'legend.fontsize': 16,'xtick.labelsize':16,'ytick.labelsize':16,'axes.labelsize':16,'font.size':16}
plt.rcParams.update(params)


In [6]:
tfilenames_FMA = []

for month in ['02','03','04']:
    if month in ['12','01','02','03','04','05','06']:
        case='BLBT02X'
    else:
        case='BLBT02'

    if month in ['07','08','09','10','11','12']:
        year='2009'
    else:
        year='2010'
    
    data_dir = '/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-'+str(case)+'-S/'
    tfilenames_month = sorted(glob.glob(data_dir+'*/eNATL60-'+str(case)+'_1h_*_gridT-2D_'+str(year)+str(month)+'??-'+str(year)+str(month)+'??.nc'))
    tfilenames_FMA=tfilenames_FMA+tfilenames_month

In [ ]:
%time dsFMA=xr.open_mfdataset(tfilenames_FMA,parallel=True)['sossheig']


In [ ]:
dsFMA_reg=dsFMA[:,1850:2150,3850:4150].chunk({'x':1,'y':1,'time_counter':2136})

In [ ]:
NPADDING = 2
%time SPtime_FMA_reg = xfft.fft(dsFMA_reg, dim='time_counter', dx=1., detrend='mean',nfft=dsFMA_reg.shape[0]*NPADDING,tapering=True)


In [ ]:
SPtime_FMA_reg

In [ ]:
%time MEANPSD_FMA=xfft.psd(SPtime_FMA_reg).mean(dim='x').mean(dim='y').load()
